In [395]:
import pandas as pd
from datetime import datetime, timedelta
import os

The below cell will Load CSVs into Pandas and create a MultiIndexed DataFrame
It will also convert Date column to datetime format.

In [396]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL']
df_list = []

for ticker in tickers:
    file_path = f"{ticker}.csv"
    df = pd.read_csv(file_path)

    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Add Ticker column if not already present
    if 'Ticker' not in df.columns:
        df['Ticker'] = ticker

    df_list.append(df)

# Concatenate and set MultiIndex
combined_df = pd.concat(df_list)
combined_df.set_index(['Ticker', 'Date'], inplace=True)


The below code will sort data by date within each ticker and handle the missing values.

In [397]:
# Sort index to ensure proper ordering
combined_df.sort_index(inplace=True)

# Interpolate missing values within each Ticker group
combined_df = combined_df.groupby(level=0).apply(lambda group: group.interpolate(method='linear')).droplevel(0)

# Forward fill for any remaining NaNs
combined_df = combined_df.groupby(level=0).apply(lambda group: group.ffill()).droplevel(0)


The below code will filter the data to include only the last 10 years

In [398]:
ten_years_ago = datetime.today() - timedelta(days=365 * 10)
combined_df = combined_df.reset_index()  # Bring 'Date' back to columns for filtering

# Filter and reapply MultiIndex
combined_df = combined_df[combined_df['Date'] >= ten_years_ago]
combined_df.set_index(['Ticker', 'Date'], inplace=True)


In [399]:
print(combined_df.columns)
print(combined_df.index.names)


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt'], dtype='object')
['Ticker', 'Date']


In [400]:
# See cleaned dataset
combined_df.head()


Open    High     Low   Close    Volume  OpenInt
Ticker Date                                                         
AAPL   2015-06-08  123.02  123.32  121.03  121.97  55195489        0
       2015-06-09  120.91  122.22  119.88  121.61  58732198        0
       2015-06-10  122.09  123.45  122.00  123.00  40948345        0
       2015-06-11  123.29  124.22  122.60  122.71  37084550        0
       2015-06-12  122.32  122.46  121.31  121.37  38579953        0

DATA TRANSFORMATION (ADD NEW COLUMNS)

In [401]:
# Step 3: Data Transformation

# Calculate daily return: % change in closing price
combined_df['Daily Return'] = combined_df.groupby(level=0)['Close'].pct_change()

# 7-day Moving Average of closing price
combined_df['7D MA'] = combined_df.groupby(level=0)['Close'].transform(lambda x: x.rolling(window=7).mean())

# 30-day Moving Average of closing price
combined_df['30D MA'] = combined_df.groupby(level=0)['Close'].transform(lambda x: x.rolling(window=30).mean())

# Rolling Volatility (30-day std deviation of returns)
combined_df['30D Volatility'] = combined_df.groupby(level=0)['Daily Return'].transform(lambda x: x.rolling(window=30).std())

# Show the result
combined_df.head(40)


Open    High     Low   Close     Volume  OpenInt  \
Ticker Date                                                             
AAPL   2015-06-08  123.02  123.32  121.03  121.97   55195489        0   
       2015-06-09  120.91  122.22  119.88  121.61   58732198        0   
       2015-06-10  122.09  123.45  122.00  123.00   40948345        0   
       2015-06-11  123.29  124.22  122.60  122.71   37084550        0   
       2015-06-12  122.32  122.46  121.31  121.37   38579953        0   
       2015-06-15  120.34  121.43  119.97  121.12   46094023        0   
       2015-06-16  121.22  122.00  120.59  121.78   33001274        0   
       2015-06-17  121.89  122.03  120.95  121.49   34493316        0   
       2015-06-18  121.43  122.44  121.42  122.03   37101639        0   
       2015-06-19  121.92  121.98  120.62  120.81   57335314        0   
       2015-06-22  121.68  122.20  121.28  121.78   35668278        0   
       2015-06-23  121.67  121.78  121.08  121.23   31717403        0   
       2015-06-24  121.41  123.87  121.32  122.25   57926303        0   
       2015-06-25  122.98  123.31  121.69  121.69   33466523        0   
       2015-06-26  121.84  122.14  120.73  120.96   46175652        0   
       2015-06-29  119.73  120.69  118.79  118.84   51514057        0   
       2015-06-30  119.83  120.36  119.16  119.70   46494077        0   
       2015-07-01  121.10  121.14  120.24  120.81   31685901        0   
       2015-07-02  120.65  120.90  120.03  120.66   28513099        0   
       2015-07-06  119.23  120.46  119.15  120.25   29403250        0   
       2015-07-07  120.15  120.39  118.11  119.95   49193474        0   
       2015-07-08  118.79  118.95  116.94  116.97   63669393        0   
       2015-07-09  118.19  118.38  113.77  114.59   82356224        0   
       2015-07-10  116.37  118.19  115.67  117.65   64290561        0   
       2015-07-13  119.32  120.01  118.63  119.92   43423667        0   
       2015-07-14  120.29  120.59  119.33  119.87   33279159        0   
       2015-07-15  119.98  121.35  119.84  121.02   35239317        0   
       2015-07-16  121.94  122.69  121.54  122.63   37955852        0   
       2015-07-17  123.19  123.72  122.44  123.72   48373947        0   
       2015-07-20  124.99  126.90  124.73  126.03   61718915        0   
       2015-07-21  126.78  126.85  124.35  124.78   80429635        0   
       2015-07-22  116.42  119.77  116.42  119.51  120975577        0   
       2015-07-23  120.43  121.29  119.35  119.45   53440014        0   
       2015-07-24  119.61  120.00  118.23  118.81   44180009        0   
       2015-07-27  117.48  117.97  116.55  117.17   46582952        0   
       2015-07-28  117.75  118.24  116.95  117.75   35226815        0   
       2015-07-29  117.54  117.87  116.68  117.39   38782874        0   
       2015-07-30  116.73  116.97  116.14  116.78   35237502        0   
       2015-07-31  117.00  117.04  115.40  115.76   44937189        0   
       2015-08-03  115.95  116.97  112.15  113.02   73291498        0   

                   Daily Return       7D MA      30D MA  30D Volatility  
Ticker Date                                                              
AAPL   2015-06-08           NaN         NaN         NaN             NaN  
       2015-06-09     -0.002952         NaN         NaN             NaN  
       2015-06-10      0.011430         NaN         NaN             NaN  
       2015-06-11     -0.002358         NaN         NaN             NaN  
       2015-06-12     -0.010920         NaN         NaN             NaN  
       2015-06-15     -0.002060         NaN         NaN             NaN  
       2015-06-16      0.005449  121.937143         NaN             NaN  
       2015-06-17     -0.002381  121.868571         NaN             NaN  
       2015-06-18      0.004445  121.928571         NaN             NaN  
       2015-06-19     -0.009998  121.615714         NaN             NaN  
       2015-06-22      0.008029  121.482857         NaN           

EXPLORATORY ANALYSIS

In [402]:
# Group by Ticker and compute average of Daily Return (excluding NaNs)
average_returns = combined_df.groupby('Ticker')['Daily Return'].mean()

# Find the stock with the highest average return
highest_avg_return_ticker = average_returns.idxmax()
highest_avg_return_value = average_returns.max()

print(f" Stock with highest average return: {highest_avg_return_ticker} ({highest_avg_return_value:.4f})")


 Stock with highest average return: AMZN (0.0017)


In [403]:
# Reset index to access Date easily
df_reset = combined_df.reset_index()

# Create Year-Month column
df_reset['YearMonth'] = df_reset['Date'].dt.to_period('M')

# Compute volatility (std of returns) per Ticker per Month
monthly_volatility = df_reset.groupby(['Ticker', 'YearMonth'])['Daily Return'].std()

# Find the highest volatility entry
most_volatile = monthly_volatility.idxmax()
most_volatile_value = monthly_volatility.max()

print(f" Most volatile month: {most_volatile[0]} in {most_volatile[1]} (Volatility = {most_volatile_value:.4f})")


 Most volatile month: TSLA in 2016-02 (Volatility = 0.0464)
